# RAG

## 1. Load DATA

In [1]:
import pandas as pd
import transformers
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
import evaluate
import numpy as np
import pandas as pd
import openpyxl
import os
from PyPDF2 import PdfReader
import glob

/home/rdecoupe/mambaforge/envs/mood_luca_review_avian_flu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-19 09:50:08.818730: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 09:50:08.874873: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-19 09:50:08.954107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 09:50:09.015682: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been re

In [2]:
file_path = './data/quantitative_value_with_context_CC.xlsx'

# Read the "missing value" worksheet
missing_value_df = pd.read_excel(file_path, sheet_name='missing value')

# Read the 'quantitative_value_with context' worksheet
quantitative_value_df = pd.read_excel(file_path, sheet_name='quantitative_value_with context')
# quantitative_value_df = quantitative_value_df.rename(columns={'Relevant context for MOOD data extraction': 'context'})

missing_value_df['source'] = 'missing_value'
quantitative_value_df['source'] = 'quantitative_value_with_context'

# drop 3 lines corresponding to a Claudia's comment
"""
Missing values not found by the script. Articles: MB7, MB8, CC6-15-17-23-32
"""
missing_value_df = missing_value_df.drop(index=range(53, 56))

# concatenate the 2 dataframe
# df = pd.concat([missing_value_df, quantitative_value_df], ignore_index=True)
df = quantitative_value_df

usable_covariates = df[df["Relevant context for MOOD data extraction"].str.lower().isin(["yes", "Yes"])]
# usable_covariates = pd.concat([missing_value_df, usable_covariates])

full_text_annotation = usable_covariates[usable_covariates["Mood extraction from Table/Figure"].str.lower().isin(["no", "No"])]
table_annotations = usable_covariates[usable_covariates["Mood extraction from Table/Figure"].str.lower().isin(["Table", "table", "table and caption"])]
figure_annotations = usable_covariates[usable_covariates["Mood extraction from Table/Figure"].str.lower().isin(["figure", "Figure", "Figure caption"])]

In [3]:
df.head(1)

,Unnamed: 0,ID,value,page,position,covariate_in_mood,covariate_found_in_text,covariate_fuzzy,context,non-standardized covariate in the context,Mood extraction from Table/Figure,Relevant context for MOOD data extraction,source
0,0,CC13,-0.4763,15,2379,Humidity Absolute spatial scale,False,False,in Figs. S3 â€“S6. Absolute humidity was foun...,Absolute humidity,No,yes,quantitative_value_with_context


In [4]:
usable_covariates[["context"]].describe()


,context
count,56
unique,55
top,variate and quadratic logistic regression Step...
freq,2


In [5]:
d = f'{full_text_annotation.iloc[0]["context"]} \t covariable found: {full_text_annotation.iloc[0]["covariate_found_in_text"]}: {(full_text_annotation.iloc[0]["non-standardized covariate in the context	"] if full_text_annotation.iloc[0]["covariate_found_in_text"] else "")}'
d

' in Figs. S3 â€“S6. Absolute humidity was found to be signiï¬\x81cantly linked to epidemic onset dates at the spatial scale ( p= 0.029), but not at the other scales. The associated coefï¬\x81cient was negative ( -0.4763). Mobility ï¬‚ows were not found to be signiï¬\x81cantly linked to epidemic onset dates (p= 0.57 with the corrected model, p= 0.73 with the uncorrected model). In the corrected model, the coefï¬\x81cient  \t covariable found: False: '

In [6]:
full_text_annotation["covariate_text"] = full_text_annotation.apply(lambda row: row["non-standardized covariate in the context"] if row["Relevant context for MOOD data extraction"] else "", axis=1)
full_text_annotation["docs"] = full_text_annotation.apply(lambda row: f'{row["context"]} \t covariable found: {row["covariate_found_in_text"]}: {row["covariate_text"]}', axis=1)

print(full_text_annotation["docs"].head(1))


0     in Figs. S3 â€“S6. Absolute humidity was foun...
Name: docs, dtype: object


/tmp/ipykernel_1160/2831026351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_text_annotation["covariate_text"] = full_text_annotation.apply(lambda row: row["non-standardized covariate in the context"] if row["Relevant context for MOOD data extraction"] else "", axis=1)
/tmp/ipykernel_1160/2831026351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_text_annotation["docs"] = full_text_annotation.apply(lambda row: f'{row["context"]} \t covariable found: {row["covariate_found_in_text"]}: {row

In [7]:
docs = full_text_annotation["docs"].tolist()
print(f"Nb of docs: {len(docs)}")
docs

Nb of docs: 20


[' in Figs. S3 â€“S6. Absolute humidity was found to be signiï¬\x81cantly linked to epidemic onset dates at the spatial scale ( p= 0.029), but not at the other scales. The associated coefï¬\x81cient was negative ( -0.4763). Mobility ï¬‚ows were not found to be signiï¬\x81cantly linked to epidemic onset dates (p= 0.57 with the corrected model, p= 0.73 with the uncorrected model). In the corrected model, the coefï¬\x81cient  \t covariable found: False: Absolute humidity',
 'tion over the course of the outbreak together with the direction of the observed transmissions. The circular correlation coef ï¬\x81cient between direction of observed transmissions and wind direction was 0.051, signi ï¬\x81cantly higher than expected when directions were uniform and independent ( P=. 0 1 ) ( Supplementary Figure 1 ). The circular correlation coef ï¬\x81cient was also signi ï¬\x81cantly higher than e \t covariable found: True: wind direction',
 'direction of the wind by chance. Using these numbers and

In [8]:
dataset_dict = DatasetDict.load_from_disk("./data/annotation_generated_from_xlsx/annotation.dataset")
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'labels'],
        num_rows: 12
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'labels'],
        num_rows: 8
    })
})

In [9]:
from difflib import SequenceMatcher
import unicodedata

train_docs_list = docs.copy()
test_docs_list = []

for i, sentence in enumerate(dataset_dict["test"]["tokens"]):
    s = " ".join(sentence)
    print(f"{i}: {s}")
    for doc in docs:
        doc_format = unicodedata.normalize("NFKD", doc.lstrip())
        if SequenceMatcher(None, doc_format.lower(), s.lower()).ratio() > 0.7:
            # print(f"{s.lower()} \n\t {doc_format.lower()}")
            # print("\n")
            # print(f"{s.lower()[0]} | {doc_format.lower()[0]}")
            if doc_format.lower()[0] == s.lower()[0]:
                print(f"\t found !: {doc}")
                train_docs_list.remove(doc)
                test_docs_list.append(doc)


0: one standard deviation ) . table 1 estimated z values for ini¬ ‚ uenza aerosols determined at low , medium , and high relative humidity rh range ( % ) estimated zvalue95 % coni¬denceinterval r 2lower upper 25a€ “ 27 0 . 29 0 . 27 0 . 31 0 . 98550a€ “ 54 0 . 27 0 . 26 0 . 31 0 . 99181a€ “ 84 0 . 22 0 . 21 0 . 23 0 . 992mcdevitt et al . 1668 aem . asm . org applied and environmental microbiology on april 1 , 2015 by university of strathclyde http : / / aem
	 found !:  one standard deviation). TABLE 1 Estimated Z values for inï¬‚uenza aerosols determined at low, medium, and high relative humidity RH range (%)Estimated Zvalue95% conï¬denceinterval R 2Lower Upper 25â€“27 0.29 0.27 0.31 0.98550â€“54 0.27 0.26 0.31 0.99181â€“84 0.22 0.21 0.23 0.992McDevitt et al. 1668 aem.asm.org Applied and Environmental Microbiology on April 1, 2015 by UNIVERSITY OF STRATHCLYDE http://aem 	 covariable found: False: Relative humidity 
1: standard deviation ) . table 1 estimated z values for ini¬ ‚ uenza 

In [10]:
len(train_docs_list)

12

## 2. Prepage RAG pipeline

In [11]:
#from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
#from langchain.chains import RetrievalQA

In [12]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/home/rdecoupe/mambaforge/envs/mood_luca_review_avian_flu/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs_for_rag = text_splitter.create_documents(train_docs_list)

db = FAISS.from_documents(docs_for_rag, embeddings)

In [14]:
retriever = db.as_retriever(search_kwargs={"k": 5})

## 3. RAG active

In [15]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain import hub
from langchain.chat_models import ChatOpenAI
from getpass import getpass

openai_api_key = getpass("OpenAI API Key: ")


In [16]:
from langchain.prompts import ChatPromptTemplate

template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context as example to help to extract covariates or risk factors from the sentence only (don't extract from the context please). If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt_template = ChatPromptTemplate.from_template(template)

rag_chain_gpt4 = (
    {"context": retriever, "question": RunnablePassthrough()}
    # | hub.pull("rlm/rag-prompt")
    | prompt_template
    # | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
    | ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)
    | StrOutputParser()
)

/home/rdecoupe/mambaforge/envs/mood_luca_review_avian_flu/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
sentence = """g10 (i.e. a 90% reduction in infectivity). These Rt values were based on the averages reported by Keeler et al. (2014) for resistance of 9 LPAIV strains in distilled, ï¬ltered and natural water (67.1, 3.1 and 30.0 days, respectively), and were also in accordance with previously published data (e.g. Stallknecht et al., 1991; Brown et al., 2009; Lebarbenchon et al., 2012 ). 2.7. The numerical model I"""

rag_chain_gpt4.invoke("Is there any covariate (or risk factor) in this following sentence: \n " + sentence)

'The sentence does not explicitly mention any covariates or risk factors.'

In [18]:
results_list = []

for sentence in test_docs_list:
    res = rag_chain_gpt4.invoke("Is there any covariate (or risk factor) in this following sentence: \n " + sentence)
    results_list.append(res)

In [19]:
results_list

['The sentence does not provide any covariates or risk factors.',
 'The sentence does not provide any covariate or risk factor.',
 'The sentence does not provide any covariate or risk factor.',
 'The covariates or risk factors in the sentence are temperature (17 or 28°C), salinity (0 or 2% NaCl), and pH levels (ranging from 6.2 to 8.2).',
 'The sentence is not in English and appears to be corrupted or improperly translated, making it impossible to extract any covariates or risk factors from it.',
 'The covariate or risk factor in the sentence is Relative Humidity (RH).',
 'The sentence does not provide any clear covariate or risk factor.',
 'The sentence does not provide any clear covariates or risk factors.']

In [20]:
test_docs_list[5]

'elettes produites artiï¬\x81ciellement Ã©tait favorisÃ©e par une RH faible [34]. De mÃªme, la transmission aÃ©rienne dâ€™une souche humaine chez le cochon dâ€™Inde Ã\xa0 20â—¦C est plus efï¬\x81cace pour une RH faible (20 Ã\xa0 35 %) que pour des valeurs plus Ã©levÃ©es (50 et 80 %) (ï¬\x81gure 3) [35]. NÃ©anmoins, certains isolats semblent Ãªtre plus stables Ã\xa0 RH Ã©levÃ©e (70 %) que pour des valeurs intermÃ©diaires (50 %) [36]. Des modÃ¨ \t covariable found: False: RH '

In [21]:
covariate_found = 0
for res in results_list:
    if res.lower().startswith("yes"):
        covariate_found += 1
print(f"Nb of covariate found: {covariate_found} | {100*covariate_found/len(test_docs_list)}")

Nb of covariate found: 0 | 0.0


In [22]:
for s, r in zip(test_docs_list, results_list):
    print(f"{s} \n \t{r}")

 one standard deviation). TABLE 1 Estimated Z values for inï¬‚uenza aerosols determined at low, medium, and high relative humidity RH range (%)Estimated Zvalue95% conï¬denceinterval R 2Lower Upper 25â€“27 0.29 0.27 0.31 0.98550â€“54 0.27 0.26 0.31 0.99181â€“84 0.22 0.21 0.23 0.992McDevitt et al. 1668 aem.asm.org Applied and Environmental Microbiology on April 1, 2015 by UNIVERSITY OF STRATHCLYDE http://aem 	 covariable found: False: Relative humidity  
 	The sentence does not provide any covariates or risk factors.
standard deviation). TABLE 1 Estimated Z values for inï¬‚uenza aerosols determined at low, medium, and high relative humidity RH range (%)Estimated Zvalue95% conï¬denceinterval R 2Lower Upper 25â€“27 0.29 0.27 0.31 0.98550â€“54 0.27 0.26 0.31 0.99181â€“84 0.22 0.21 0.23 0.992McDevitt et al. 1668 aem.asm.org Applied and Environmental Microbiology on April 1, 2015 by UNIVERSITY OF STRATHCLYDE http://aem.asm. 	 covariable found: False: Relative humidity  
 	The sentence does 

In [23]:
t = hub.pull("rlm/rag-prompt")
t

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

### Same with GPT3.5

In [24]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    # | hub.pull("rlm/rag-prompt")
    | prompt_template
    | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
    # | ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)
    | StrOutputParser()
)

In [25]:
results_list_gpt3_5 = []

for sentence in test_docs_list:
    res = rag_chain.invoke("Is there any covariate (or risk factor) in this following sentence: \n " + sentence)
    results_list_gpt3_5.append(res)

In [26]:
for s, r in zip(test_docs_list, results_list_gpt3_5):
    print(f"{s} \n \t{r}")

 one standard deviation). TABLE 1 Estimated Z values for inï¬‚uenza aerosols determined at low, medium, and high relative humidity RH range (%)Estimated Zvalue95% conï¬denceinterval R 2Lower Upper 25â€“27 0.29 0.27 0.31 0.98550â€“54 0.27 0.26 0.31 0.99181â€“84 0.22 0.21 0.23 0.992McDevitt et al. 1668 aem.asm.org Applied and Environmental Microbiology on April 1, 2015 by UNIVERSITY OF STRATHCLYDE http://aem 	 covariable found: False: Relative humidity  
 	Covariate found: Relative humidity, Absolute humidity, Wind-borne transmission.
standard deviation). TABLE 1 Estimated Z values for inï¬‚uenza aerosols determined at low, medium, and high relative humidity RH range (%)Estimated Zvalue95% conï¬denceinterval R 2Lower Upper 25â€“27 0.29 0.27 0.31 0.98550â€“54 0.27 0.26 0.31 0.99181â€“84 0.22 0.21 0.23 0.992McDevitt et al. 1668 aem.asm.org Applied and Environmental Microbiology on April 1, 2015 by UNIVERSITY OF STRATHCLYDE http://aem.asm. 	 covariable found: False: Relative humidity  
 	

In [27]:
results_list_gpt3_5

['Covariate found: Relative humidity, Absolute humidity, Wind-borne transmission.',
 'Covariate found: False: Relative humidity, Absolute humidity, wind-borne, distance, transmission related to wind, filtered water.',
 'Covariate found: False: Relative humidity, Absolute humidity, transmission related to wind, wind-borne, distance, natural water.',
 'Covariate found: salinity\nCovariate found: temperature\nCovariate found: natural water',
 'Covariate found: True: wind direction\nCovariate found: False: salinity\nCovariate found: False: Absolute humidity',
 'The covariate found in the sentence is RH (relative humidity).',
 "I don't know.",
 'Covariate found: True: wind direction\nCovariate found: False: Absolute humidity\nCovariate found: False: AH (absolute humidity)']

## 4. Infer on all diseases

In [28]:
from grobid.tei import Parser
import os
import json

In [54]:
tei_files_path = "./data/grobid/"

tei_file_example = "./data/grobid/CC10 Aerosol Susceptibility of Influenza Virus to UV-C Light.pdf.tei.xml"

# list_of_diseases = ["chikungunya", "leptospirosi", "influenza" ]
list_of_diseases = ["influenza" ]
sections_to_keep = ["abstract", "results", "discussion"]
chunk_size = 256

list_of_base_models = ["GPT-3.5", "GPT-4"]
# list_of_base_models = ["GPT-4"]
# list_of_base_models = ["GPT-3.5"]


In [52]:
def chunk_relevant_sections(tei_file):
    chunks = []
    md = {
        "title": tei_file.split("/")[-1],
        "id": (tei_file.split("/")[-1]).split(" ")[0],
    }

    # load PDF into python dict
    with open(tei_file, "rb") as xml_file:
        xml_content = xml_file.read()
    parser = Parser(xml_content)
    article = parser.parse()
    article = json.loads(article.to_json())

    # work on abstract
    abstract = article["abstract"]
    for p, paragraph in enumerate(abstract["paragraphs"]):
        for i in range(0, len(paragraph["text"]), chunk_size):
            # print(f"{i} : {(i)} | {i + chunk_size -1}")
            chunk_with_md = md.copy()
            chunk_with_md["section"] = "abstract"
            chunk_with_md["paragraph_nb"] = p
            chunk_with_md["chunk_nb"] = i/chunk_size
            chunk_with_md["text"] = paragraph["text"][i:i+chunk_size-1]
            chunks.append(chunk_with_md)

    # work on usefull sections
    for s in article["sections"]:
        if (s["title"].lower() in sections_to_keep):
            for p, paragraph in enumerate(s["paragraphs"]):
                for i in range(0, len(paragraph["text"]), chunk_size):
                    # print(f"{i} : {(i)} | {i + chunk_size -1}")
                    chunk_with_md = md.copy()
                    chunk_with_md["section"] = s["title"]
                    chunk_with_md["paragraph_nb"] = p
                    chunk_with_md["chunk_nb"] = i/chunk_size
                    chunk_with_md["text"] = paragraph["text"][i:i+chunk_size-1]
                    chunks.append(chunk_with_md)
    
    return chunks

def extract_covariates(text):
    # Split the text into lines
    lines = text.split('\n')
    # Extract covariates using list comprehension
    covariates = [line.split(': ')[1].strip().rstrip('.') for line in lines if 'Covariate found:' in line or 'No covariate found:' in line]
    # Join the covariates into a single string
    return [', '.join(covariates)]


In [55]:
for disease in list_of_diseases:
    tei_files_path_disease = tei_files_path + "/" + disease

    list_of_chunk = []
    for doc in os.listdir(tei_files_path_disease):
        list_of_chunk.extend(chunk_relevant_sections( os.path.join(tei_files_path_disease, doc)))
    df = pd.DataFrame(list_of_chunk)

    for pretrained_model in list_of_base_models:
        print(f"Work on: {pretrained_model}")
        results_list_infer_gpt3_5 = []

        for sentence in df["text"]:
        # for sentence in df.iloc[0:3]["text"]:
            if pretrained_model == 'GPT-3.5':
                res = rag_chain.invoke("Is there any covariate (or risk factor) in this following sentence: \n " + sentence)
            else:
                res = rag_chain_gpt4.invoke("Is there any covariate (or risk factor) in this following sentence: \n " + sentence + "please provide only a python list like '[temperature, humidity] without any explanation'")
            results_list_infer_gpt3_5.append(res)
        
        df[pretrained_model] = pd.DataFrame(results_list_infer_gpt3_5)
        if pretrained_model == 'GPT-3.5':#need to parse because GPT-3.5 doest not follow well the instructions
            df[pretrained_model] = df[pretrained_model].apply(extract_covariates)
    
    df.to_csv(f"./data/whole_inference_llm_{disease}.csv")



Work on: GPT-3.5
Work on: GPT-4


In [ ]:
# df[pretrained_model] = df[pretrained_model].apply(extract_covariates)
# df.to_csv(f"./data/whole_inference_llm_{disease}.csv")